In [62]:
import jieba
import jieba.analyse
import pandas as pd
from pandas import DataFrame
import numpy as np

In [107]:
df = pd.read_csv('coms1.csv',encoding = "gbk")

'评价方未及时做出评价,系统默认好评!'
'此用户没有填写评论!'
评价方未及时做出评价,系统默认好评!

In [108]:
txt1 = ''.join(df['rateContent']).replace('此用户没有填写评论!','').replace('评价方未及时做出评价,系统默认好评!','')

In [110]:
txt1

'记录仪安装车上，很上档次很好看，送了很多东西，哈哈，主要是记录仪太清晰了，比我原来的清楚多了，晚上都很清楚，从车外面看高端大气，里面也很上档次，客服很耐心的教，这是淘宝最满意的一次购物，大家不要再犹豫了，速度买，绝对是你们需要的行车记录仪安装好，线隐蔽成功很美观，开机使用只有高清导航，经咨询升级后电子狗，ADAS都能用了，真的不错，也比较清晰，满意！宝贝不错，就是打开应用有点延时，这个价位应该可以。宝贝收到，还没有安装，看着可以，满意像素还可以，，功能都打开后非常的卡顿，根本就不能用收到宝贝。真的很不错。物流特别快。摄像头特别清晰。看起来特别高大上。最满意的一次网购东西很好也很清晰，功能也很全，很实用，像素也很好，晚上也看的清楚其它的功能都不错，可惜少了个移动侦测，有时候大屏不显示，要拔钥匙重开。东西挺好，比原来车上的清楚多了！操作也比较简单！给个赞！自己动手十分钟搞定！非常不错，挺好的，画面非常清晰，试了试，挺好，安装简便给爸爸买的，收到后，就连接使用了，爸爸很满意质量很好物美价廉，清晰度很高没有这个adas吧，挺想要的，有了吧，一直喊车道偏离，挺烦的&hellip;就这样吧，现在感觉也没太大用处，变道吧也喊车道偏离&hellip;确实感觉有些烦，关了昨天自己亲手安装挺好安装的，东西挺好的，价格实惠，视频清晰，推荐大家购买功能强大，图像清晰，非常满意！设计简洁实用，摄像头清晰，比我想象的要好！先好评！待后（不是我天天用！）追评！摄像清晰买过很多次了，老板值得信赖客服很好，东西装好了，感觉不错，用几天再来评价感谢卖家耐心的指点，装好了。画面很清楚，安装跟简单，客服服务很好！影像很清晰功能强大 耐用就好不挡视线，视频清晰，希望耐用好用很清晰比以前那个好多了   上图看看商家对买家考虑的服务非常周到  买家非常满意 5分什么破东西，出来的图像还是变形的，而且一点都不清楚，很不错，就是有点太大画质还行，可以接受，毕竟价格在那里，功能也还可以，主要可以放在后视镜后面，这点非常好白天还行，晚上效果一般，有光就一团白的，车牌细节就几乎被-团光给遮掉了，另外个头可不小图片是夜间的效果，还不错，功能齐全，就是在运行的时候有时会黑屏，经过咨询内存卡使用前要格式化，总体来说还不错，这个价位算可以的了！自己安装前后摄像头，如果不弄倒车影像，不难。最难是SUV后摄像头走线过后盖连接处防水

In [111]:
seg_list = jieba.cut(txt1,cut_all=False)

In [119]:
print("分词结果:")
#print("/".join(seg_list))
#获取关键词
tags=jieba.analyse.extract_tags(txt1,topK=50)
print("关键词:")
print(" ".join(tags))

分词结果:
关键词:
不错 记录仪 安装 清晰 非常 功能 满意 好评 宝贝 效果 收到 客服 卖家 清晰度 行车 电子狗 摄像头 可以 东西 导航 语音 质量 服务态度 清楚 好用 值得 方便 购买 耐心 店家 老板 晚上 服务 使用 物流 追评 评价 hellip 机子 发货 录像 朋友 功能强大 就是 感觉 画面 实用 很快 ADAS 后视镜


In [120]:
type(tags)

list

In [133]:
from collections import Counter
import jieba.analyse
import time

In [139]:
data = jieba.posseg.cut(txt1)
data = dict(Counter(data))

In [141]:
%ls

 驱动器 C 中的卷没有标签。
 卷的序列号是 AE9D-1BB4

 C:\Users\GWT\LearningJupyter\jupyterfiles\crawler\taobao 的目录

2018/07/09  13:54    <DIR>          .
2018/07/09  13:54    <DIR>          ..
2018/07/09  11:30    <DIR>          .ipynb_checkpoints
2018/07/05  15:57            49,011 commentv0.1.ipynb
2018/07/06  11:35           180,519 commentv0.2.ipynb
2018/07/09  13:52         1,688,686 commentv0.3.ipynb
2018/07/09  12:16                21 coms.csv
2018/07/09  12:13         1,348,273 coms1.csv
2018/07/09  13:54         1,585,041 jieba.ipynb
2018/07/06  13:50             1,842 print.ipynb
2018/07/06  13:41         6,564,374 评论.csv
               8 个文件     11,417,767 字节
               3 个目录 16,887,365,632 可用字节


In [143]:
%cd c:/Users/GWT/LearningJupyter/jupyterfiles/

c:\Users\GWT\LearningJupyter\jupyterfiles


In [144]:
%ls

 驱动器 C 中的卷没有标签。
 卷的序列号是 AE9D-1BB4

 c:\Users\GWT\LearningJupyter\jupyterfiles 的目录

2018/07/04  10:20    <DIR>          .
2018/07/04  10:20    <DIR>          ..
2018/07/03  10:12    <DIR>          .ipynb_checkpoints
2018/05/21  12:02             8,979 1.csv
2018/05/21  11:59               103 2.csv
2018/05/24  16:04         6,235,209 30.csv
2018/05/21  15:04            10,723 c.csv
2018/07/04  17:41    <DIR>          crawler
2018/05/30  16:27         6,130,919 d.csv
2018/06/25  10:53    <DIR>          data
2018/06/05  13:09            17,405 data.csv
2018/06/05  13:22            17,405 data2222.csv
2018/04/04  17:37            23,192 echarts.ipynb
2018/04/04  17:33             3,939 echartsmap.ipynb
2018/03/01  10:46             2,341 echartstimeline.ipynb
2017/12/27  15:19        31,814,355 foods-2011-10-03.json
2018/05/03  11:18               424 Insurance.csv
2018/06/05  13:25            60,505 itchat.ipynb
2018/05/03  14:21            13,028 matplotlib.ipynb
2003/03/26  15:18       